# Setup a reinforcement learning problem for a virtual particle image velocimetry (PIV) setup

***

In [ ]:
from pykitPIV.ml import PIVEnv
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
import numpy as np
import onnxruntime
from collections import deque
import torch
import sys, os

We have five actions:

In [ ]:
n_actions = 5

Prepare specifications for pykitPIV parameters:

In [ ]:
particle_spec = {'diameters': (1, 1),
                 'distances': (2, 2),
                 'densities': (0.4, 0.4),
                 'diameter_std': 1,
                 'seeding_mode': 'random'}

flowfield_spec = {'flowfield_size': (200, 500),
                  'flowfield_type': 'random smooth',
                  'gaussian_filters': (10, 10),
                  'n_gaussian_filter_iter': 10,
                  'displacement': (2, 2),
                  # 'apply_SLM': True,
                  # 'integral_time_scale': 1,
                  # 'sigma': 1,
                  # 'n_stochastic_particles': 1000000,
                  # 'n_iterations': 100
                 }

motion_spec = {'n_steps': 10,
               'time_separation': 1,
               'particle_loss': (0, 0),
               'particle_gain': (0, 0)}

image_spec = {'exposures': (0.98, 0.98),
              'maximum_intensity': 2**16-1,
              'laser_beam_thickness': 1,
              'laser_over_exposure': 1,
              'laser_beam_shape': 0.95,
              'alpha': 1/8,
              'clip_intensities': True,
              'normalize_intensities': False}

Use LIMA as an inference model:

In [ ]:
class ONNXmodel:
    
    def __init__(self,
                 onnx_file_path):
        
        self.name = "ONNX"
        self.providers = ['CPUExecutionProvider']
        self.session = onnxruntime.InferenceSession(onnx_file_path, 
                                                    None,
                                                    providers=self.providers)

        self.input_name = self.session.get_inputs()[0].name  
        print('Input Name:', self.input_name)   
 
    def inference(self, x):
        
        output = self.session.run([], {self.input_name:x/np.max(x)})[0] 
      
        return output

    def empty(self):
        
         with torch.no_grad():
            torch.cuda.empty_cache()

In [ ]:
model_file = '../Lima_L4_PAD_SR2_dyn.onnx'
print("model:", model_file, '  exist:', os.path.exists(model_file))

In [ ]:
lima_inference_model = ONNXmodel(model_file)

***

## Create the RL environment

Initialize the `Gymnasium` environment:

In [ ]:
env = PIVEnv(interrogation_window_size=(100,100),
             interrogation_window_size_buffer=10,
             particle_spec=particle_spec,
             motion_spec=motion_spec,
             image_spec=image_spec,
             flowfield_spec=flowfield_spec,
             user_flowfield=None,
             inference_model=lima_inference_model,
             random_seed=10)

Reset the environment:

In [ ]:
camera_position, prediction_tensor, targets_tensor = env.reset()

In [ ]:
plt = env.render(camera_position,
                 c='white',
                 s=20,
                 lw=1,
                 normalize_cbars=True,
                 cmap=cmc.roma,
                 add_streamplot=True,
                 streamplot_density=3,
                 streamplot_color='k',
                 streamplot_linewidth=0.3,
                 figsize=(15,6), 
                 filename='ml_PIVEnv_render.png')

In [ ]:
for _ in range(0,20):
    camera_position, reward, terminated, truncated = env.step(1, 
                                                              verbose=False)

In [ ]:
plt = env.render(camera_position,
                 c='white',
                 s=20,
                 lw=1,
                 normalize_cbars=True,
                 cmap=cmc.roma,
                 add_streamplot=True,
                 streamplot_density=3,
                 streamplot_color='k',
                 streamplot_linewidth=0.3,
                 figsize=(15,6), 
                 filename='ml_PIVEnv_render.png')

Reset the environment with a user-defined initial camera position:

In [ ]:
camera_position, prediction_tensor, targets_tensor = env.reset(imposed_camera_position=np.array([0, 50]))

plt = env.render(camera_position,
                 c='white',
                 s=20,
                 lw=1,
                 normalize_cbars=True,
                 cmap=cmc.roma,
                 add_streamplot=True,
                 streamplot_density=3,
                 streamplot_color='k',
                 streamplot_linewidth=0.3,
                 figsize=(15,6), 
                 filename='ml_PIVEnv_render.png')

***

## Create an RL agent

Define the Q-network:

In [ ]:
class QNetwork(tf.keras.Model):
    
    def __init__(self, n_actions):
        
        super(QNetwork, self).__init__()
        
        self.dense1 = tf.keras.layers.Dense(10, activation='linear', kernel_initializer=tf.keras.initializers.Ones)
        self.dense2 = tf.keras.layers.Dense(10, activation='linear', kernel_initializer=tf.keras.initializers.Ones)
        self.output_layer = tf.keras.layers.Dense(n_actions, activation='linear', kernel_initializer=tf.keras.initializers.Ones)

    def call(self, state):
        
        x = self.dense1(state)
        x = self.dense2(x)
        
        return self.output_layer(x)

Initialize the camera agent:

In [ ]:
image_obj = env.record_particles(camera_position)

In [ ]:
plt.imshow(image_obj.images_I1[0,0,:,:], cmap='Greys_r', origin='lower')

In [ ]:
plt.imshow(image_obj.images_I2[0,0,:,:], cmap='Greys_r', origin='lower')

In [ ]:
image_obj.plot_field_magnitude(0, 
                               add_streamplot=True,
                               streamplot_density=1,
                               streamplot_color='k',)

In [ ]:
image_obj.animate_image_pair(0, filename='image.gif')

***